#TopEditor - traitements NER et enrichissements des textes TEI

Le dossier du projet se trouve ici :
```
/content/drive/MyDrive/Colab Notebooks/TopEditor/
```
Description des sous-dossiers associés aux traitements:
* **NERmodels** : les modèles NER produits de reperage d'entités nommées (et autres)

* **data/TEI/Final/input**: fichiers TEI stylé via Word customisé Metopes et converti en schéma métopes (via XMLMind)

* **data/TEI/Final/clean**: fichiers TEI du dossier input legerement "nettoyés" manuellement du bruit produit par Word après consignes Métopes

* **data/TEI/Final/tagged**: fichiers TEI avec balisage automatisé via un modèle NER custom

* **data/TEI/Final/reference_pers**: fichiers tableurs avec les identifiants personnes à désambiguiser manuellement

* **data/TEI/Final/reference_place**: fichiers tableurs avec les identifiants place (sens restreint) à désambiguiser manuellement

* **data/TEI/Final/tagged/updated_person**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes

* **data/TEI/Final/tagged/updated_person_upated_place**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes et pour les lieux

* **data/TEI/Final/tagged_corrected_Sara**: fichiers TEI dans updated_person_upated_place avec la correction manuelle par Sara via Oxygène des balises fautives

* **data/TEI/Final/tagged_corrected_Sara/TagAugmented** fichiers dans tagged_corrected_Sara avec un balisage automatisé par reglès des dates déjà reperées par le modèle NER et pas du tout repérées




In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Controler aucune perte de paragraphes entre traitements auto (à faire systématiquement)

Comparer les listes d'identifiants des paragraphes par fichier pour controler aucune perte d'information avant et après traitements

### Check intégrité identifiants des paragraphes

In [6]:
import os
from lxml import etree

def get_paragraph_ids_from_file(file_path):
    """
    Extrait les identifiants xml:id des balises <p> dans un fichier XML TEI.

    Args:
        file_path (str): Chemin du fichier XML.

    Returns:
        set: Ensemble des identifiants xml:id trouvés.
    """
    namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}
    paragraph_ids = set()

    try:
        tree = etree.parse(file_path)
        root = tree.getroot()

        # Récupérer tous les paragraphes <p>
        paragraphs = root.findall('.//tei:p', namespaces=namespace)
        for p in paragraphs:
            p_id = p.get('{http://www.w3.org/XML/1998/namespace}id')  # Récupérer xml:id
            if p_id:
                paragraph_ids.add(p_id)

    except etree.XMLSyntaxError as e:
        print(f"Erreur XML dans {file_path}: {e}")

    return paragraph_ids


def compare_paragraph_ids_by_file(folder1, folder2, suffix_fold2, prefix_fold2):
    """
    Compare les identifiants des balises <p> fichier par fichier entre deux dossiers XML TEI.

    Args:
        folder1 (str): Chemin du premier dossier XML.
        folder2 (str): Chemin du deuxième dossier XML.

    Returns:
        dict: Dictionnaire avec les différences par fichier.
            - 'missing_in_folder2': Identifiants présents uniquement dans folder1
            - 'missing_in_folder1': Identifiants présents uniquement dans folder2
            - 'in_both': Identifiants présents dans les deux fichiers
    """
    results = {}

    # Liste des fichiers XML dans chaque dossier
    files1 = {f for f in os.listdir(folder1) if f.endswith('.xml')}
    print(files1)
    files2 = {f for f in os.listdir(folder2) if f.endswith('.xml')}

    if suffix_fold2 and ".xml" in suffix_fold2: # is suffix
      files2 = {f.replace(suffix_fold2, ".xml") for f in files2}
    if prefix_fold2: #is prefix
      files2 = {f.replace(prefix_fold2, "") for f in files2}
    print(files2)

    # Trouver les fichiers communs
    #common_files = files1 & files2
    common_files = files1.union(files2)
    #print(common_files)
    for file_name in sorted(common_files):  # Trier pour un affichage propre
        file1_path = os.path.join(folder1, file_name)

        #file_name_2 = file_name
        if suffix_fold2 and ".xml" in suffix_fold2: # is suffix
          file_name = file_name.replace(".xml", suffix_fold2)
        if prefix_fold2: #is prefix
          file_name = str(prefix_fold2) + file_name
        file2_path = os.path.join(folder2, file_name)

        print(f"\n📂 Fichier de comparaison : {file_name}")
        print(f"📄 Fichier 1 : {file1_path}")
        print(f"📄 Fichier 2 : {file2_path}")
        print("🔍 Recherche des différences...")
        ids_1 = get_paragraph_ids_from_file(file1_path)
        ids_2 = get_paragraph_ids_from_file(file2_path)

        # Comparer les identifiants
        results[file_name] = {
            'missing_in_folder2': ids_1 - ids_2,  # Présents uniquement dans file1
            'missing_in_folder1': ids_2 - ids_1,  # Présents uniquement dans file2
            'in_both': ids_1 & ids_2  # Présents dans les deux
        }

    return results

### Check intégrité contenu des paragraphes

Fonction pour vérifier l'intégrité des paragraphes avant et après traitement: le premier et le dernier mots doivent être les mêmes entre avant et après, le nombre de mots et des balises ne doivent pas être perdues.

In [7]:
import csv
import lxml.etree as ET
from pathlib import Path

namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

def extract_paragraphs_with_first_last_words(xml_file):
    """
    Extrait les paragraphes <p> d'un fichier XML avec leurs identifiants, premier et dernier mot.
    Retourne un dictionnaire {id_p: (premier_mot, dernier_mot)}
    """
    parser = ET.XMLParser(remove_blank_text=True)
    tree = ET.parse(xml_file, parser)
    root = tree.getroot()

    paragraphs_data = {}

    for paragraph in root.findall(".//tei:p", namespaces=namespace):
        p_id = paragraph.get("{http://www.w3.org/XML/1998/namespace}id")
        #print(p_id)
        text_content = "".join(paragraph.itertext()).strip()  # Récupérer tout le texte

        if text_content:
            words = text_content.split()
            first_word = words[0]
            last_word = words[-1]
            word_count = len(words)
            tag_count = sum(1 for _ in paragraph.iter())

            paragraphs_data[p_id] = (first_word, last_word, word_count, tag_count)

    return paragraphs_data

def compare_paragraphs_between_versions(input_folder, output_folder, prefix_fold2, suffix_fold2, report_csv):
    """
    Compare les paragraphes entre les versions d'un fichier XML dans deux dossiers.
    Vérifie leur présence et compare les premiers/derniers mots.
    Sauvegarde un rapport CSV.
    """
    input_path = Path(input_folder)
    output_path = Path(output_folder)
    print(input_path)
    print(output_path)
    with open(report_csv, mode="w", newline="", encoding="utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([
            "Fichier", "ID_Paragraphe", "Présence",
            "1er Mot (Entrée)", "1er Mot (Sortie)",
            "Dernier Mot (Entrée)", "Dernier Mot (Sortie)",
            "Nb mots (Entrée)", "Nb mots (Sortie)", "Nb mots Identique",
            "Nb balises (Entrée)", "Nb balises (Sortie)", "Nb balises Identique"
        ])

        for input_file in input_path.glob("*.xml"):
            print(input_file)
            if suffix_fold2 and ".xml" in suffix_fold2: # is suffix
              output_file = output_path / f"{input_file.name}".replace(".xml", f"{suffix_fold2}")  # Ajout du suffix
            if prefix_fold2: #is prefix
              output_file = output_path / f"{prefix_fold2}{input_file.name}"  # Ajout du prefix

            print(output_file)
            if not output_file.exists():
                print(f"⚠️ Fichier absent dans le dossier de sortie : {output_file.name}")
                continue

            input_paragraphs = extract_paragraphs_with_first_last_words(input_file)
            output_paragraphs = extract_paragraphs_with_first_last_words(output_file)

            all_paragraph_ids = set(input_paragraphs.keys()) | set(output_paragraphs.keys())
            #print(all_paragraph_ids)

            for p_id in all_paragraph_ids:
                #print(p_id)
                in_input = p_id in input_paragraphs
                in_output = p_id in output_paragraphs

                if in_input and in_output:
                    first_in, last_in, words_in, tags_in = input_paragraphs[p_id]
                    first_out, last_out, words_out, tags_out = output_paragraphs[p_id]
                    same_word_count = "Oui" if words_in == words_out else "Non"
                    same_tag_count = "Oui" if tags_in == tags_out else "Non"

                    csv_writer.writerow([input_file.name, p_id, "OK",
                                         first_in, first_out,
                                         last_in, last_out,
                                         words_in, words_out, same_word_count,
                                         tags_in, tags_out, same_tag_count])
                elif in_input:
                    first_in, last_in, words_in, tags_in = input_paragraphs[p_id]
                    csv_writer.writerow([input_file.name, p_id, "Manquant en sortie",
                                         first_in, "", last_in, "",
                                         words_in, "", "Non",
                                         tags_in, "", "Non"])
                else:
                    first_out, last_out, words_out, tags_out = output_paragraphs[p_id]
                    csv_writer.writerow([input_file.name, p_id, "Nouveau en sortie",
                                         "", first_out, "", last_out,
                                         "", words_out, "Non",
                                         "", tags_out, "Non"])

    print(f"✅ Rapport généré : {report_csv}")


## Anotaciones NER avec ORYX (Test)

In [ ]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/Oryx/input/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/Oryx/output"

differences_by_file = compare_paragraph_ids_by_file(folder1, folder2, suffix_fold2='_oryx.xml')

for file, diffs in differences_by_file.items():
    print(f"\n📂 Comparaison pour {file}:")
    print(f"🔴 Manquants dans {folder2}: {diffs['missing_in_folder2']}")
    print(f"🟢 Manquants dans {folder1}: {diffs['missing_in_folder1']}")
    print(f"✅ Présents dans les deux: {diffs['in_both']}")

compare_paragraphs_between_versions(
    folder1,
    folder2,
    '_oryx.xml', "rapport_paragraphes_oryx.csv"
)

{'2combiné_T2 clj. San Pedro_clean.xml'}
{'2combiné_T2 clj. San Pedro_clean.xml'}

📂 Fichier de comparaison : 2combiné_T2 clj. San Pedro_clean.xml
📄 Fichier 1 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/Oryx/input/2combiné_T2 clj. San Pedro_clean.xml
📄 Fichier 2 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/Oryx/output/2combiné_T2 clj. San Pedro_clean_oryx.xml
🔍 Recherche des différences...

📂 Comparaison pour 2combiné_T2 clj. San Pedro_clean.xml:
🔴 Manquants dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/Oryx/output: set()
🟢 Manquants dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/Oryx/input/: set()
✅ Présents dans les deux: {'p28', 'p120', 'p319', 'p274', 'p135', 'p66', 'p665', 'p739', 'p524', 'p706', 'p245', 'p587', 'p156', 'p821', 'p705', 'p294', 'p372', 'p844', 'p807', 'p642', 'p281', 'p296', 'p473', 'p178', 'p881', 'p502', 'p582', 'p546', 'p419', 'p608', 'p740', 'p231', 'p257', 'p360', 'p8

On perd parfois le balisage d'origine autour des notes de bas de page, par exemple:

DE:
```
740 mrs. 8 d. de vieille monnaie<hi
rend="sup"
><note
n="174"
place="foot"
xml:id="ftn174"
><p
xml:id="p741"
>. OF 1080, f° 43.</p
></note
></hi
>.</p
>
```
A:
```
<en en-type="modo_pago">
740 mrs. 8 d. de vieille monnaie<hi rend="sup">
<note n="174" place="foot" xml:id="ftn174">
<p xml:id="p741"/>.
OF 1080, f° 43.</note>
</hi></en>.</p>
```






## Anotaciones NER (clean -> tagged) Sans ORYX

**TROIS PROBLEMES ICI**,

VOIR CONTROLE

il est necessaire de relancer le traitement sur 2,4,5 (et sur l'ensemble) et vérifier avec les controles:

* on perd des paragraphes ! la correction a été faite dans le code (il me semble) mais revérifier avec les controles

* on perd les notes de bas de page, à restituer
```
<p
rend="TEI_localparaDate"
xml:id="p10"
>CH_BO_00. En 1372, le chanoine Alvar López tient la maison que tenait auparavant la sœur de l’archiprêtre d’Ocaña, depuis le 15/8/1371 (era 409), pour deux ans, au prix de 72 mrs. par an<hi
rend="sup"
><note
n="3"
place="foot"
xml:id="ftn3"
><p
xml:id="p11"
>. OF 929, f° 28 r°.</p
></note
></hi
>.</p
```

* certains paragraphes n'ont pas le meme decompte des mots (en lien avec le problème précedant)


**ATTENTION**:
Pour ne pas perdre le travail de correction de Sara: regenérer les fichiers 2,4,5 sans erreur (en vérifiant les controles) et refaire sur ces nouvelles version à la main toutes ses corrections via Oxygène.
il est possible de reinjecter les paragraphes manquants mais impossible pour les notes de bas de pages pour tous les paragraphes


### Check ids paragraphes

In [ ]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/clean/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/"
differences_by_file = compare_paragraph_ids_by_file(folder1, folder2, prefix_fold2="semtags_", suffix_fold2='')

for file, diffs in differences_by_file.items():
    print(f"\n📂 Comparaison pour {file}:")
    print(f"🔴 Manquants dans {folder2}: {diffs['missing_in_folder2']}")
    print(f"🟢 Manquants dans {folder1}: {diffs['missing_in_folder1']}")
    print(f"✅ Présents dans les deux: {diffs['in_both']}")


{'2combiné_T2 clj. San Pedro_clean.xml'}
{'2combiné_T2 clj. San Pedro_clean.xml'}

📂 Fichier de comparaison : semtags_2combiné_T2 clj. San Pedro_clean.xml
📄 Fichier 1 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/clean/2combiné_T2 clj. San Pedro_clean.xml
📄 Fichier 2 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean.xml
🔍 Recherche des différences...

📂 Comparaison pour semtags_2combiné_T2 clj. San Pedro_clean.xml:
🔴 Manquants dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/: {'p752', 'p737', 'p902', 'p743', 'p805', 'p153', 'p196', 'p13', 'p588', 'p807', 'p890', 'p22', 'p637', 'p797', 'p762', 'p784', 'p332', 'p460', 'p594', 'p37', 'p30', 'p318', 'p157', 'p457', 'p624', 'p163', 'p142', 'p590', 'p813', 'p873', 'p736', 'p189', 'p336', 'p795', 'p859', 'p267', 'p791', 'p416', 'p144', 'p904', 'p51', 'p823', 'p407', 'p863', 'p155', 'p312', 'p418', 'p406', 'p478', 'p600', 'p263

### Check contenu paragraphes

In [14]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/clean/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/"

compare_paragraphs_between_versions(
    folder1,
    folder2,
    suffix_fold2="", prefix_fold2="semtags_", report_csv="rapport_paragraphes_ner.csv"
)

/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/clean/2combiné_T2 clj. San Pedro_clean.xml
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean.xml
✅ Rapport généré : rapport_paragraphes_ner.csv


## Personnes (tagged -> reference_pers, updated_person)


### Check ids paragraphes

In [22]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person"
differences_by_file = compare_paragraph_ids_by_file(folder1, folder2, prefix_fold2="", suffix_fold2='_updated_person.xml')

for file, diffs in differences_by_file.items():
    print(f"\n📂 Comparaison pour {file}:")
    print(f"🔴 Manquants dans {folder2}: {diffs['missing_in_folder2']}")
    print(f"🟢 Manquants dans {folder1}: {diffs['missing_in_folder1']}")
    print(f"✅ Présents dans les deux: {diffs['in_both']}")


{'semtags_2combiné_T2 clj. San Pedro_clean.xml'}
{'semtags_2combiné_T2 clj. San Pedro_clean.xml'}

📂 Fichier de comparaison : semtags_2combiné_T2 clj. San Pedro_clean_updated_person.xml
📄 Fichier 1 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean.xml
📄 Fichier 2 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person/semtags_2combiné_T2 clj. San Pedro_clean_updated_person.xml
🔍 Recherche des différences...

📂 Comparaison pour semtags_2combiné_T2 clj. San Pedro_clean_updated_person.xml:
🔴 Manquants dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person: set()
🟢 Manquants dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/: set()
✅ Présents dans les deux: {'p860', 'p413', 'p698', 'p464', 'p224', 'p1', 'p300', 'p487', 'p728', 'p648', 'p864', 'p788', 'p240', 'p276', 'p17', 'p119', 'p808', 'p384', 'p262', 'p674', 'p394', 'p526', 

### Check contenu paragraphes

In [32]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person"

compare_paragraphs_between_versions(
    folder1,
    folder2,
    prefix_fold2="", suffix_fold2="_updated_person.xml", report_csv="rapport_paragraphes_pers.csv"
)

/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean.xml
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person/semtags_2combiné_T2 clj. San Pedro_clean_updated_person.xml
✅ Rapport généré : rapport_paragraphes_pers.csv


## Lieux - collaciones, calles, CTMB (tagged -> reference_place, updated_person_updated_place)

### Check ids paragraphes

In [8]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person_updated_place"
differences_by_file = compare_paragraph_ids_by_file(folder1, folder2, suffix_fold2='_updated_person_updated_place.xml', prefix_fold2="")

for file, diffs in differences_by_file.items():
    print(f"\n📂 Comparaison pour {file}:")
    print(f"🔴 Manquants dans {folder2}: {diffs['missing_in_folder2']}")
    print(f"🟢 Manquants dans {folder1}: {diffs['missing_in_folder1']}")
    print(f"✅ Présents dans les deux: {diffs['in_both']}")

{'semtags_2combiné_T2 clj. San Pedro_clean.xml'}
{'semtags_2combiné_T2 clj. San Pedro_clean.xml'}

📂 Fichier de comparaison : semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place.xml
📄 Fichier 1 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean.xml
📄 Fichier 2 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person_updated_place/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place.xml
🔍 Recherche des différences...

📂 Comparaison pour semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place.xml:
🔴 Manquants dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person_updated_place: set()
🟢 Manquants dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/: set()
✅ Présents dans les deux: {'p91', 'p511', 'p206', 'p274', 'p200', 'p439', 'p601', 'p627', 'p209', 'p672', 'p852', 'p409', 'p313', 

### Check contenu paragraphes

In [11]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person_updated_place"

compare_paragraphs_between_versions(
    folder1,
    folder2,
    prefix_fold2="", suffix_fold2="_updated_person_updated_place.xml", report_csv="rapport_paragraphes_places.csv"
)

/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person_updated_place
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean.xml
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/updated_person_updated_place/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place.xml
✅ Rapport généré : rapport_paragraphes_places.csv


## Dates (tagged_corrected_Sara -> TagAugmented)

### Check ids paragraphes

In [12]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented"
differences_by_file = compare_paragraph_ids_by_file(folder1, folder2, prefix_fold2='', suffix_fold2='_enrichCarmen.xml')

for file, diffs in differences_by_file.items():
    print(f"\n📂 Comparaison pour {file}:")
    print(f"🔴 Manquants dans {folder2}: {diffs['missing_in_folder2']}")
    print(f"🟢 Manquants dans {folder1}: {diffs['missing_in_folder1']}")
    print(f"✅ Présents dans les deux: {diffs['in_both']}")

{'semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara.xml'}
{'semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara.xml'}

📂 Fichier de comparaison : semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara_enrichCarmen.xml
📄 Fichier 1 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara.xml
📄 Fichier 2 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara_enrichCarmen.xml
🔍 Recherche des différences...

📂 Comparaison pour semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara_enrichCarmen.xml:
🔴 Manquants dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented: set()
🟢 Manquants dans /content/drive/M

### Check contenu paragraphes

In [14]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented"

compare_paragraphs_between_versions(
    folder1,
    folder2,
    suffix_fold2 = '_enrichCarmen.xml', prefix_fold2 ='', report_csv="rapport_paragraphes_dates.csv"
)

/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara.xml
/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara_enrichCarmen.xml
✅ Rapport généré : rapport_paragraphes_dates.csv


## Check pertes de notes (tagged_corrected_Sara vs clean)

Recuperer les ids de paragraphes p des fichiers originaux afin de les restituer sur les versions corrigées par Sara en mars 2025 (cela ne concerne que les textes 2,4,5,6) - FAILED

In [ ]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/clean"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/"
differences_by_file = compare_paragraph_ids_by_file(folder1, folder2,
                            suffix_fold2='_updated_person_updated_place_corrSara.xml',
                            prefix_fold2='semtags_')

for file, diffs in differences_by_file.items():
    print(f"\n📂 Comparaison pour {file}:")
    print(f"🔴 Manquants dans {folder2}: {diffs['missing_in_folder2']}")
    print(f"🟢 Manquants dans {folder1}: {diffs['missing_in_folder1']}")
    print(f"✅ Présents dans les deux: {diffs['in_both']}")

{'5combiné_T5 La Puerta del Fierro_clean.xml', '2combiné_T2 clj. San Pedro_clean.xml', '4combiné_T4 La Calle de los Azacanes_clean.xml', '6combiné_T6 San Marcos_clean.xml'}
{'5combiné_T5 La Puerta del Fierro_clean.xml', '2combiné_T2 clj. San Pedro_clean.xml', '4combiné_T4 La Calle de los Azacanes_clean.xml', '6combiné_T6 San Marcos_clean.xml'}

📂 Fichier de comparaison : semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara.xml
📄 Fichier 1 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/clean/2combiné_T2 clj. San Pedro_clean.xml
📄 Fichier 2 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara.xml
🔍 Recherche des différences...

📂 Fichier de comparaison : semtags_4combiné_T4 La Calle de los Azacanes_clean_updated_person_updated_place_corrSara.xml
📄 Fichier 1 : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Fin

A partir des identifiants des paragraphes manquants dans les fichiers de clean, on les injecte dans les fichiers de tagged_corrected_Sara - ne fonctionne pas (peu gérable pour 4 fichiers, faire à la main).

après reinjection vérifier que les ordres de grandeur dans les différences

In [ ]:
folder1 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/clean"
folder2 = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/"

compare_paragraphs_between_versions(
    folder1,
    folder2,
    '_enrichCarmen.xml', "rapport_paragraphes_pertes.csv"
)

## OLD Agregar las informaciones tematicas al documento TEI

A partir de los documentos XML-TEI tratados por un proceso de NER, hacemos post-tratamientos para agregar informacion tematica por ejemplo los identificadores de las casas, aqui es agregar los tags <idno> a los codigos casas en los textos.

Ojo: hay que preservar los tags de las EN agregadas en la etapa previa.

**Tipo de informacion por parrafo**

Caso 1 : identificador de casa solo en un parrafo "p" dentro de una seccion de textos "div". Ejemplo:
```
 <p rend="TEI_localMcode" xml:id="p11">CH_AO_00</p>
```

Caso 2 : identificador de casa en un parrafo "p" al principio del texto seguido de una fecha immediata. Ejemplo:

```
 <p rend="TEI_localparaDate" xml:id="p19">CH_AO_00. En 1412, Diego Fern... </p>
```

Caso 3 : identificador de casa en un parrafo "p" al principio del texto con una fecha no immediata. Ejemplo:

```
 <p rend="TEI_localparaDonnees" xml:id="p23">CH_AO_00. La maison disparaît après 1419, pour dégager la cathédrale ... </p>
```





In [ ]:
import os
import pandas as pd
from lxml import etree

# Chemins d'accès
gdrive_path = "/content/drive/MyDrive/Colab Notebooks/TopEditor"
dossier_textes = gdrive_path + '/data/TEI/'
csv_file_path = gdrive_path + '/data/Textenrichment/TopEditor_codigos_casas_con_URL.csv'

# Namespace TEI
namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

# Lire les codes et les URL à partir du fichier CSV
def lire_codes_csv(csv_file_path, colonne_index_code=0, colonne_index_url=8):
    df = pd.read_csv(csv_file_path, encoding='utf-8')
    codes_urls = {}

    for _, row in df.iterrows():
        # Vérification si la colonne code est NaN ou vide
        code = str(row.iloc[colonne_index_code]).strip()
        if code and code != "nan":  # Exclure les valeurs NaN ou vides
            url = row.iloc[colonne_index_url] if len(row) > colonne_index_url else None
            codes_urls[code] = url

    return codes_urls

def nettoyer_texte_p(texte):
    if texte.startswith('.') or texte.startswith(' '):
        texte = texte.lstrip('. ').lstrip()  # Enlève les points et espaces du début
    return texte

# Fonction pour ajouter un attribut `corresp` à l'élément <p> si un code est trouvé au début du texte
def ajouter_corresp_si_code_trouve(p, codes_urls):
    if p.text:
        for code, url in codes_urls.items():
            if p.text.startswith(code):
                # Ajouter ou mettre à jour l'attribut `corresp` de <p>
                p.attrib['corresp'] = "TOPEDITORIndexMaisons.xml#"+code
                #p.text = p.text[len(code):].lstrip() A FAIRE ?
                p.text = nettoyer_texte_p(p.text)
                return code, url  # Retourner le code et l'URL trouvés
    return None, None

# Fonction pour générer un fichier d'index des maisons
def generer_index_maisons(codes_urls_maisons, fichier_sortie):
    # Enregistrer le namespace xml pour xml:id et xml:lang
    etree.register_namespace('xml', 'http://www.w3.org/XML/1998/namespace')

    # Créer la racine <listPlace> avec le namespace xml pour xml:id
    list_place = etree.Element("listPlace", {"{http://www.w3.org/XML/1998/namespace}id": "indexMaisons"})

    # Ajouter des descriptions multilingues avec le bon namespace pour xml:lang
    descriptions = {
        "en": "The list of identified houses in medieval Toledo.",
        "es": "La lista de casas identificadas en el Toledo medieval.",
        "fr": "La liste des maisons identifiées dans le Tolède médiéval."
    }

    for lang, text in descriptions.items():
        desc = etree.SubElement(list_place, "desc", {"{http://www.w3.org/XML/1998/namespace}lang": lang})
        desc.text = text

    # Ajouter chaque maison trouvée comme <place> avec un attribut `url`
    for code, url in codes_urls_maisons.items():
        place = etree.SubElement(list_place, "place", {"{http://www.w3.org/XML/1998/namespace}id": code})
        place_name = etree.SubElement(place, "placeName")
        place_name.text = code

        # Ajouter l'URL à l'élément <place>
        if url:
            place.attrib["ref"] = url

    # Sauvegarder le fichier XML
    tree = etree.ElementTree(list_place)
    tree.write(fichier_sortie, encoding="UTF-8", xml_declaration=True, pretty_print=True)
    print(f"Fichier d'index des maisons sauvegardé sous : {fichier_sortie}")

# Lire les codes et les URL depuis le fichier CSV
codes_urls = lire_codes_csv(csv_file_path)

# Collecter tous les codes de maisons trouvés
codes_urls_maisons_trouves = {}

# Lire et traiter les fichiers XML
for fichier in os.listdir(dossier_textes):
    if fichier.startswith('semtags_TOPEDITOR-1-Lequartierdeschanoines') and fichier.endswith('.xml'):
        chemin_fichier = os.path.join(dossier_textes, fichier)

        # Charger et parser le fichier XML avec lxml
        tree = etree.parse(chemin_fichier)
        root = tree.getroot()

        # Trouver le body
        body = root.find('.//tei:body', namespaces=namespace)

        # Extraire tous les éléments <p> ayant un attribut rend
        paragraphs = body.xpath('.//tei:p[@rend]', namespaces=namespace)

        # Ajouter l'attribut `corresp` si nécessaire
        for p in paragraphs:
            code_trouve, url_trouve = ajouter_corresp_si_code_trouve(p, codes_urls)
            if code_trouve:
                codes_urls_maisons_trouves[code_trouve] = url_trouve

        # Sauvegarder le fichier XML modifié
        sortie_fichier = os.path.join(dossier_textes, 'codesmaison_' + fichier)
        tree.write(sortie_fichier, encoding="UTF-8", xml_declaration=True, pretty_print=True)

        print(f"Fichier {fichier} traité et sauvegardé sous {sortie_fichier}.")

# Générer le fichier d'index des maisons avec les URL
fichier_index_sortie = os.path.join(dossier_textes, 'TOPEDITORIndexMaisons.xml')
generer_index_maisons(codes_urls_maisons_trouves, fichier_index_sortie)


Fichier semtags_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml traité et sauvegardé sous /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/codesmaison_semtags_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml.
Fichier d'index des maisons sauvegardé sous : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/TOPEDITORIndexMaisons.xml
